### Install libtorrent and Initialize Session
It is not recommended to edit this script anywhere other than Google Colabs since it's the only place you can test the file reliably.

In [ ]:
#@markdown <h2><b>⬅ Run This Cell to Install Torrent Requirements</b></h2>
!pip install lbry-libtorrent
!pip install python-libtorrent-bin==1.2.12
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 17.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 29.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libtorrent-rasterbar9
Suggested packages:
  libtorrent-rasterbar-dbg
The following NEW packages will be installed:
  libtorrent-rasterbar9 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 27 not upgraded.
Need to get 1,569 kB of archives.
After this operation, 5,718 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtorrent-ra

In [ ]:
#@markdown <h2><b>⬅ Run This Cell to  Mount Gdrive</b></h2>
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Add From Torrent File
You can run this cell to add more files as many times as you want

In [ ]:
from google.colab import files
Drive = "ms_media" #@param {type:"string"}
Path = "" #@param {type:"string"}
Save_To = "/content/drive/Shareddrives/" + Drive + "/" + Path

source = files.upload()
params = {
    "save_path": "$Save_To",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

KeyboardInterrupt: ignored

### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [ ]:
#@markdown <h2><b>⬅ Run this Cell to add magnetic links</b></h2>
Save_Path = "/content/drive/Shareddrives/ms_media" #@param {type:"string"}
params = {"save_path": "$Save_Path"}

while True:
    magnet_link = input("Enter Magnet Link Or Type 'exit': ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

Enter Magnet Link Or Type 'exit': magnet:?xt=urn:btih:56ea2f5252d230c876b3ddd5fc1c4aa590f641fc&dn=www.1TamilMV.cfd%20-%20Ponniyin%20Selvan%201%20(2022)%20Tamil%20TRUE%20WEB-DL%20-%201080p%20-%20AVC%20-%20(DD%2b5.1%20-%20640Kbps%20%26%20AAC)%20-%203GB%20-%20ESub.mkv&tr=udp%3a%2f%2ftracker.openbittorrent.com%3a80%2fannounce&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2ftracker.openbittorrent.com%3a80%2fannounce&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=http%3a%2f%2ftracker.trackerfix.com%3a80%2fannounce&tr=udp%3a%2f%2f9.rarbg.me%3a2850%2fannounce&tr=udp%3a%2f%2f9.rarbg.to%3a2970%2fannounce&tr=udp%3a%2f%2ftracker.thinelephant.org%3a12780%2fannounce&tr=udp%3a%2f%2ftracker.slowcheetah.org%3a14760%2fannounce
Enter Magnet Link Or Type 'exit': exit


### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [ ]:
#@title
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…